In [4]:
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-large-instruct")
persist_directory = "../database/anwesha_chroma"
vectorstore = Chroma(
    persist_directory=persist_directory, embedding_function=embeddings)
retriever = vectorstore.as_retriever()

C:\Users\mdsaz\AppData\Local\Temp\ipykernel_13580\2632607041.py:7: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


In [1]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
examples = [
    {
        "input": "অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে?",
        "output": "শম্ভুনাথ",
    },
    {
        "input": "কাকে অনুপমের ভাগ্য দেবতা বলে উল্লেখ করা হয়েছে?",
        "output": "মামাকে",
    },
]
# We now transform these to example messages
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert at HSC26-Bangla1st-Paper textbook. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:""",
        ),
        # Few shot examples
        few_shot_prompt,
        # New question
        ("user", "{question}"),
    ]
)

In [3]:
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter
from langchain_groq import ChatGroq
# LLM
llm = ChatGroq(
    model="moonshotai/kimi-k2-instruct",
)

generate_queries_step_back = prompt | ChatGroq(
    model="moonshotai/kimi-k2-instruct",
) | StrOutputParser()
question = "অনুপম তার মামার চেয়ে কত বছরের ছোট ছিল?"
generate_queries_step_back.invoke({"question": question})

'অনুপম তার মামার চেয়ে দশ বছরের ছোট ছিল।'

In [5]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
# Response prompt
response_prompt_template = """You are an expert of HSC26-Bangla1st-Paper textbook. I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant. Otherwise, ignore them if they are not relevant.

# {normal_context}
# {step_back_context}

# Original Question: {question}
# Answer:"""
response_prompt = ChatPromptTemplate.from_template(response_prompt_template)

chain = (
    {
        # Retrieve context using the normal question
        "normal_context": RunnableLambda(lambda x: x["question"]) | retriever,
        # Retrieve context using the step-back question
        "step_back_context": generate_queries_step_back | retriever,
        # Pass on the question
        "question": lambda x: x["question"],
    }
    | response_prompt
    | ChatGroq(
        model="moonshotai/kimi-k2-instruct",
    )
    | StrOutputParser()
)

chain.invoke({"question": question})

'‘অপরিচিতা’ গল্প অনুযায়ী, অনুপম তার মামার চেয়ে **ছয় বছরের** ছোট ছিল।'